In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("train.csv")

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.describe()

,id,label
count,20800.000000,20800.000000
mean,10399.500000,0.500625
std,6004.587135,0.500012
min,0.000000,0.000000
25%,5199.750000,0.000000
50%,10399.500000,1.000000
75%,15599.250000,1.000000
max,20799.000000,1.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [6]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
df=df.fillna('')

In [8]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [9]:
df.columns

Index(['id', 'title', 'author', 'text', 'label'], dtype='object')

In [10]:
df=df.drop(['id', 'title', 'author'], axis=1)

In [11]:
df.head()

,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,Ever get the feeling your life circles the rou...,0
2,"Why the Truth Might Get You Fired October 29, ...",1
3,Videos 15 Civilians Killed In Single US Airstr...,1
4,Print \nAn Iranian woman has been sentenced to...,1


In [12]:
from nltk.corpus import stopwords

In [13]:
from nltk.stem.porter import PorterStemmer

In [14]:
import re

In [15]:
port_stem=PorterStemmer()

In [16]:
port_stem.stem("Hi thIs is FARHAN * % %@@@")

'hi this is farhan * % %@@@'

In [17]:
def stemming(content):
    con=re.sub('[^a-zA-Z]', ' ', content)
    con=con.lower()
    con=con.split()
    con=[port_stem.stem(word) for word in con if not word in stopwords.words('english')]
    con=' '.join(con)
    return con

In [18]:
stemming('Hi this is Farhan')

'hi farhan'

In [19]:
df['text']= df['text'].apply(stemming)

In [20]:
x=df['text']

In [21]:
y=df['label']

In [23]:
y.shape

(20800,)

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
x_train , x_test , y_train, y_test = train_test_split(x, y, test_size=0.20)

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
vect=TfidfVectorizer()

In [28]:
x_train=vect.fit_transform(x_train)
x_test=vect.transform(x_test)

In [29]:
x_test.shape

(4160, 98927)

In [30]:
from sklearn.tree import DecisionTreeClassifier

In [31]:
model=DecisionTreeClassifier()

In [32]:
model.fit(x_train, y_train)

DecisionTreeClassifier()

In [33]:
prediction=model.predict(x_test)

In [34]:
prediction

array([0, 1, 0, ..., 1, 0, 1], dtype=int64)

In [35]:
model.score(x_test, y_test)

0.8769230769230769

In [36]:
import pickle

In [37]:
pickle.dump(vect, open('vector.pkl', 'wb'))

In [38]:
pickle.dump(model, open('model.pkl', 'wb'))

In [39]:
vector_form=pickle.load(open('vector.pkl', 'rb'))

In [40]:
load_model=pickle.load(open('model.pkl', 'rb'))

In [41]:
def fake_news(news):
    news=stemming(news)
    input_data=[news]
    vector_form1=vector_form.transform(input_data)
    prediction = load_model.predict(vector_form1)
    return prediction

In [48]:
val=fake_news(""" Sunday on MSNBCâ€™s â€œAM Joy,â€ actor and director Rob Reiner accused President Donald Trump of being â€œmentally unstable. â€  Reiner said, â€œItâ€™s scary, itâ€™s scary, because if you look at the capacity that cyber warfare has, and if you know about Stuxnet, which some people do and donâ€™t know about that, they were able to weaponize, you know, a cyber warfare and actually go in and blow up centrifuges in Iran.  Thatâ€™s what scares me. We donâ€™t have, anybody at the helm of our country right now. He is  â€”   you know, I hate to say it, we donâ€™t want to use the word mentally ill, but he is mentally unstable, this man, and so who is going to take charge here when the next attack. â€ â€œIf the first attack, which was going into our elections, which we know happened, if it had been that a bomb blew up in New York, everybody would be going crazy, he continued. â€œThatâ€™s what happened. He blew up something. And now he could blow more stuff up. So we have to be  â€”   you know, weâ€™ve got to make sure that our country is stable. And right now itâ€™s not stable. â€ Follow Pam Key on Twitter @pamkeyNEN """)

In [49]:
if val==[0]:
    print('REAL NEWS')
else:
    print('FAKE NEWS')

REAL NEWS
